In [1]:
# %cd D:\IIIT\LAB_SE\ADR\LLM4ADR\0_shot\Decision_0_shot
%cd codes/SE/ADR/LLM4ADR/Finetuning/Code

/home2/rudra.dhar/codes/SE/ADR/LLM4ADR/Finetuning/Code


In [3]:
pip install tokenizers==0.13.3


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from evaluate import load
import pandas as pd
import nltk
from BARTScore.bart_score import BARTScorer

In [3]:
# CACHE_DIR = '/scratch/adyansh/cache'
# CACHE_DIR = 'D:/IIIT/LAB_SE/ADR/cache'
CACHE_DIR = '/scratch/rudra.dhar/cache'
MODEL_NAME = 'babbage-002'
nltk.data.path.append(CACHE_DIR)

In [4]:
rouge = load('rouge', cache_dir=CACHE_DIR)
bleu = load('bleu', cache_dir=CACHE_DIR)
meteor = load('meteor', cache_dir=CACHE_DIR)
bertscore = load("bertscore", cache_dir=CACHE_DIR)
bartscore = BARTScorer(device='cuda:1', cache_dir=CACHE_DIR)

[nltk_data] Downloading package wordnet to
[nltk_data]     /home2/rudra.dhar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home2/rudra.dhar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home2/rudra.dhar/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [6]:
# data = pd.read_csv(f'../results/{MODEL_NAME}.csv')
data = pd.read_json(f'../results/{MODEL_NAME}.jsonl', lines=True)
data.rename(columns={'Prediction': 'babbage-002'}, inplace=True)
print(len(data))
data.head()

1049


,Context,Decision,babbage-002
0,"## Context\nThe Identity, Metadata, and Storag...",## Decision\nIn order to achieve One of the en...,## Decision\nWe will use [AWS Amplify](https:/...
1,## Context\nWe cannot design endpoints in a fl...,## Decision\nWe have decided to go with option...,## Decision\nWe'll use a nested structure for ...
2,## Context\nReal world assembly programming ex...,## Decision\nTo prevent code from messing up t...,## Decision\nWe will not support the stack poi...
3,## Context\nContract-first approach enforces a...,## Decision\n[OpenAPI Specification](https://s...,## Decision\nWe will use a contract-first appr...
4,## Context and Problem Statement\nKnow it befo...,"## Decision\nChosen option: `Sentry`, because ...",## Decision\nWe will use [Flaky](https://flaky...


In [7]:
# remove rows in df where MODEL_NAME column is empty
data = data.dropna(subset=[MODEL_NAME])
len(data)

1049

In [8]:
results = {}

results['rouge'] = rouge.compute(predictions=data[MODEL_NAME],references= data['Decision'])
print('Rouge Done')
results['bleu'] = bleu.compute(predictions=data[MODEL_NAME],references= data['Decision'])
print('Bleu Done')
results['meteor'] = meteor.compute(predictions=data[MODEL_NAME],references= data['Decision'])
print('Meteor Done')
results['bertscore'] = bertscore.compute(predictions=data[MODEL_NAME],references= data['Decision'], lang='en', batch_size = 64)
print('BertScore Done')
results['bartscore_recall'] = bartscore.score(data[MODEL_NAME].tolist(), data['Decision'].tolist(), batch_size = 4)
results['bartscore_precision'] = bartscore.score(data['Decision'].tolist(), data[MODEL_NAME].tolist(), batch_size = 4)

Rouge Done
Bleu Done
Meteor Done
BertScore Done


In [10]:
cols = ['precision', 'recall', 'f1']

for c in cols:
    results['bertscore'][c] = pd.Series(results['bertscore'][c]).mean()
results['bartscore_recall'] = pd.Series(results['bartscore_recall']).mean()
results['bartscore_precision'] = pd.Series(results['bartscore_precision']).mean()
results['bartscore_F1'] = (results['bartscore_recall'] + results['bartscore_precision'])/2

In [11]:
results

{'rouge': {'rouge1': 0.18279206557006913,
  'rouge2': 0.07701318593929313,
  'rougeL': 0.14784910558305314,
  'rougeLsum': 0.14745435924792386},
 'bleu': {'bleu': 0.08872388308212627,
  'precisions': [0.19156559144692553,
   0.09196789295615448,
   0.06491709000287549,
   0.05418129465385563],
  'brevity_penalty': 1.0,
  'length_ratio': 1.4938467307561192,
  'translation_length': 262923,
  'reference_length': 176004},
 'meteor': {'meteor': 0.23481211356305676},
 'bertscore': {'precision': 0.8017927238893463,
  'recall': 0.8486827199406801,
  'f1': 0.8237019557402859,
  'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.24.0)'},
 'bartscore_recall': -4.000075102335391,
 'bartscore_precision': -2.187544419438187,
 'bartscore_F1': -3.0938097608867885}

In [13]:
# write results to a json file
import json
with open(f'../metrics/{MODEL_NAME}.json', 'w') as f:
    json.dump(results, f, indent=4)